In [25]:
import requests
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# API для получения эмбеддингов
def get_embeddings(texts):
    JINA_API_KEY = "jina_aab7734c727a44ec8283de5c8868aaacu5rPL9aESbUboLpx4u4cyk-hTGOM"
    url = 'https://api.jina.ai/v1/embeddings'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {JINA_API_KEY}'
    }
    data = {
        'input': texts,
        'model': 'jina-embeddings-v3',
        'task': 'classification',
        'dimensions': 1024
    }
    response = requests.post(url, headers=headers, json=data)
    return np.array([item['embedding'] for item in response.json()['data']])

# Пример текстов для классификации
texts = [
    "Этот продукт отличный, я очень доволен!",
    "Ужасный опыт, больше не куплю.",
    "Сервис был на высоте, рекомендую всем.",
    "Цена слишком высокая для такого качества.",
    "боль и разочарование",
    "фигня какая-то, вообще не понятная шляпа",
    "Спасибо,понравилось",
    "очень доволен, лучшая покупка",
]

# Метки классов (0 - негативный, 1 - позитивный)
labels = [1, 0, 1, 0, 0, 0, 1, 1]

# Получаем эмбеддинги для текстов
embeddings = get_embeddings(texts)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Инициализируем модель CatBoostClassifier
model = CatBoostClassifier(iterations=100, depth=4, learning_rate=0.1, loss_function='Logloss', verbose=False)

# Обучаем модель на тренировочных данных
model.fit(X_train, y_train)

# Прогнозирование на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy * 100:.2f}%')

# Прогнозирование классов для новых текстов
new_texts = [
    "Это лучший продукт, который я когда-либо покупал.",
    "Плохое качество, не советую.",
    "Отличный сервис,быстрая доставка",
    "высокая цена",
    "разочарован не советую",
    "не понравилось",
    "полная фигня",
    "не рекомендую",
    "все супер,спасибо",
    "товар понравился,буду заказывать еще",
    "товар пришел плохого качества",
    
]

# Получаем эмбеддинги новых текстов
new_embeddings = get_embeddings(new_texts)

new_predictions = model.predict(new_embeddings)

# Подсчет количества позитивных и негативных отзывов
positive_count = sum(1 for pred in new_predictions if pred == 1)
negative_count = sum(1 for pred in new_predictions if pred == 0)
total=len(new_predictions)
positive_percentage = (positive_count / total) * 100 if total > 0 else 0
negative_percentage = (negative_count / total) * 100 if total > 0 else 0

# Вывод количества отзывов
print(f'Количество позитивных отзывов: {positive_count}')
print(f'Количество негативных отзывов: {negative_count}')
print(f'Всего отзывов: {total}')
print(f'Процент позитивных отзывов: {positive_percentage:.2f}%')
print(f'Процент негативных отзывов: {negative_percentage:.2f}%')

# Вывод результата
for text, pred in zip(new_texts, new_predictions):
    print(f"Текст: {text} - Класс: {'Позитивный' if pred == 1 else 'Негативный'}")

Точность модели: 100.00%
Количество позитивных отзывов: 4
Количество негативных отзывов: 7
Всего отзывов: 11
Процент позитивных отзывов: 36.36%
Процент негативных отзывов: 63.64%
Текст: Это лучший продукт, который я когда-либо покупал. - Класс: Позитивный
Текст: Плохое качество, не советую. - Класс: Негативный
Текст: Отличный сервис,быстрая доставка - Класс: Позитивный
Текст: высокая цена - Класс: Негативный
Текст: разочарован не советую - Класс: Негативный
Текст: не понравилось - Класс: Негативный
Текст: полная фигня - Класс: Негативный
Текст: не рекомендую - Класс: Негативный
Текст: все супер,спасибо - Класс: Позитивный
Текст: товар понравился,буду заказывать еще - Класс: Позитивный
Текст: товар пришел плохого качества - Класс: Негативный
